In [1]:
# Import necessary modules
from crewai_tools import PDFSearchTool, tool
from crewai import Agent, Crew, Task
from langchain_ollama import Ollama
from langchain_community.tools.tavily_search import TavilySearchResults

# Use a local LLM like Ollama
llm = Ollama(model="llama2")

# Load a PDF document into PDFSearchTool
pdf_tool = PDFSearchTool(pdf="sample_document.pdf")

# Define a web search tool for comparison
web_search_tool = TavilySearchResults()

# Define a simple Router Tool
@tool
def simple_router_tool(question: str) -> str:
    """Routes queries to the appropriate resource."""
    if "document" in question.lower():
        return "pdf"
    else:
        return "web"

# Define Router Agent
router_agent = Agent(
    role="Router",
    goal="Route the question to PDF or Web Search based on context.",
    backstory=(
        "You are a routing assistant. You decide whether a query is about a document "
        "or requires a web search."
    ),
    llm=llm,
)

# Define Router Task
router_task = Task(
    description="Analyze the question and decide whether it should be routed to PDF or Web Search.",
    expected_output="Output 'pdf' for document queries or 'web' for web search queries.",
    tools=[simple_router_tool],
    agent=router_agent,
)

# Define Retriever Agent
retriever_agent = Agent(
    role="Retriever",
    goal="Retrieve information from the appropriate source.",
    backstory=(
        "You are a retriever. Use the specified tool to fetch answers from either a PDF or web source."
    ),
    llm=llm,
)

# Define Retriever Task
retriever_task = Task(
    description="Based on the router's output, retrieve the answer from the appropriate tool.",
    expected_output="Retrieve relevant text as the answer.",
    context=[router_task],
    tools=[pdf_tool, web_search_tool],
    agent=retriever_agent,
)

# Create the Crew
simple_rag_crew = Crew(
    agents=[router_agent, retriever_agent],
    tasks=[router_task, retriever_task],
    verbose=True,
)

# Test Input
inputs = {"question": "What does the document say about machine learning?"}

# Run the Crew
response = simple_rag_crew.run(inputs)
print("\n=== Final Output ===")
print(response)


ImportError: cannot import name 'Ollama' from 'langchain_ollama' (/opt/anaconda3/envs/ths/lib/python3.12/site-packages/langchain_ollama/__init__.py)